In [2]:
from datasets import COCODateset
from config import active_config

In [ ]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

class CaptionPreprocessor():
    def __init__(self, clean_descriptions=None):
        self.tokenizer = Tokenizer()
        self.clean_descriptions = clean_descriptions or active_config.clean_descriptions
        
    def fit_captions(captions_list):
        if self.clean_descriptions:
            captions_list = self.description_cleaner(captions_list)
        captions_list = self.eos_adder(captions_list)
        self.tokenizer.fit_on_texts(captions_list)
        
    def description_cleaner(self, captions_list):
        # TODO clean descriptions
        return captions_list
        
    def eos_adder(self, captions_list):
        return [caption + ' ' + self.EOS_TOKEN for caption in captions_list]

In [8]:
class DataProvider():
    def __init__(self, best_caption=False):
        self.dataset = COCODateset(best_caption)
        self.caption_preprocessor = CaptionPreprocessor()
        self.best_caption = best_caption
        self.build()
    
    def build(self):
        training_set = dataset.training_set
        if best_caption:
            #TODO map all captions corresponding to all images in a list
        else:
            #TODO map all captions in a list

IndentationError: expected an indented block (<ipython-input-8-934841c5cb38>, line 12)